# ð JetX Model EÄitimi - Google Colab (Fixed v7.3)

**6 Model Ensemble Sistemi ile JetX tahmin modellerini eÄitin - Dependency Conflict Fixed!**

---

## ð v7.3 Yenilikler

- ð§ **Dependency Conflict Fixed**: Torch/Torchaudio versiyon uyumluluÄu Ã§Ã¶zÃ¼ldÃ¼
- â¡ **Optimized Installation**: SÄ±ralÄ± ve gÃ¼venli paket kurulumu
- ð¡ï¸ **Version Locking**: TÃ¼m kritik paketlerin versiyonlarÄ± sabitlendi
- ð **Clean Installation**: ÃakÄ±Åan paketlerin temizlenmesi

In [ ]:
import subprocess
import sys
import os
import json
import time
from datetime import datetime

print("="*80)
print("ð¦ HAZIRLIK - 6 Model Ensemble Sistem v7.3 (DEPENDENCY FIXED)")
print("ð§ Torch/Torchaudio Conflict Resolved")
print("="*80)

start_time = time.time()

# Google Drive Bağlantısı
print("\n📁 Google Drive bağlanıyor...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive başarıyla bağlandı!")
    
    drive_path = '/content/drive/MyDrive/JetX_Models_v7_3'
    os.makedirs(drive_path, exist_ok=True)
    print(f"✅ Drive klasörü oluşturuldu: {drive_path}")
except Exception as e:
    print(f"⚠️ Drive bağlantı hatası: {e}")
    drive_path = None

# ÖNCE ÇAKIŞAN PAKETLERİ TEMİZLE
print("\n🧹 Önce eski paketler temizleniyor...")

# PyTorch ve ilgili paketleri tamamen temizle
pytorch_packages = ['torch', 'torchaudio', 'torchvision', 'pytorch-tabnet']
for package in pytorch_packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", package], 
                      check=False, capture_output=True)
        print(f"   - {package} temizlendi")
    except:
        pass

# TensorFlow ve ilgili paketleri temizle (çakışmaları önlemek için)
tensorflow_packages = ['tensorflow', 'tensorflow-gpu']
for package in tensorflow_packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", package], 
                      check=False, capture_output=True)
        print(f"   - {package} temizlendi")
    except:
        pass

print("✅ Paket temizleme tamamlandı")

# TEMEL PAKETLERİ ÖNCE YÜKLE (PyTorch bağımsız)
print("\n📦 Temel paketler yükleniyor...")
basic_packages = [
    'numpy>=1.24.0',
    'pandas>=2.0.0', 
    'scipy>=1.10.0',
    'scikit-learn>=1.3.0',
    'joblib>=1.3.0',
    'matplotlib>=3.7.0',
    'seaborn>=0.12.0',
    'pyyaml>=6.0',
    'python-dateutil>=2.8.0',
    'tqdm>=4.65.0'
]

for package in basic_packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      check=True)
        print(f"   ✅ {package}")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ {package}: {e}")

# PYTORCH EKOSİSTEMİNİ DOĞRU SIRAYLA YÜKLE
print("\n🔥 PyTorch ekosistemi yükleniyor (Fixed Versions)...")

# CUDA kontrolü
try:
    import torch
    if torch.cuda.is_available():
        cuda_version = torch.version.cuda
        print(f"   🎯 CUDA {cuda_version} tespit edildi")
        
        if cuda_version == "12.1":
            torch_index = "https://download.pytorch.org/whl/cu121"
            torch_version = "2.2.0+cu121"
            torchaudio_version = "2.2.0+cu121"
        elif cuda_version == "12.0":
            torch_index = "https://download.pytorch.org/whl/cu120"
            torch_version = "2.2.0+cu120"
            torchaudio_version = "2.2.0+cu120"
        elif cuda_version == "11.8":
            torch_index = "https://download.pytorch.org/whl/cu118"
            torch_version = "2.1.0+cu118"
            torchaudio_version = "2.1.0+cu118"
        else:
            # Varsayılan olarak CUDA 11.8 kullan
            torch_index = "https://download.pytorch.org/whl/cu118"
            torch_version = "2.1.0+cu118"
            torchaudio_version = "2.1.0+cu118"
    else:
        # CPU only
        torch_index = "https://download.pytorch.org/whl/cpu"
        torch_version = "2.1.0+cpu"
        torchaudio_version = "2.1.0+cpu"
        print("   💻 CUDA bulunamadı, CPU versiyonu kullanılacak")
        
except ImportError:
    # Torch henüz yüklü değil, varsayılan CUDA 11.8 kullan
    torch_index = "https://download.pytorch.org/whl/cu118"
    torch_version = "2.1.0+cu118"
    torchaudio_version = "2.1.0+cu118"
    print("   🔵 Varsayılan CUDA 11.8 kullanılacak")

# PyTorch kurulumu
try:
    subprocess.run([
        sys.executable, "-m", "pip", "install", "-q", 
        f"torch=={torch_version}",
        f"torchaudio=={torchaudio_version}",
        "--index-url", torch_index
    ], check=True)
    print(f"   ✅ PyTorch {torch_version}")
    print(f"   ✅ TorchAudio {torchaudio_version}")
except subprocess.CalledProcessError as e:
    print(f"   ❌ PyTorch kurulumu başarısız: {e}")
    print("   🔄 CPU versiyonu deneniyor...")
    try:
        subprocess.run([
            sys.executable, "-m", "pip", "install", "-q", 
            "torch==2.1.0+cpu",
            "torchaudio==2.1.0+cpu",
            "--index-url", "https://download.pytorch.org/whl/cpu"
        ], check=True)
        print("   ✅ PyTorch CPU versiyonu yüklendi")
    except:
        print("   ❌ PyTorch kurulumu tamamen başarısız")

# PyTorch tabnet kurulumu
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pytorch-tabnet==4.1.0"], 
                  check=True)
    print("   ✅ PyTorch TabNet")
except subprocess.CalledProcessError as e:
    print(f"   ❌ TabNet: {e}")

# TENSORFLOW KURULUMU (PyTorch'dan sonra)
print("\n🧠 TensorFlow yükleniyor...")
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "tensorflow==2.14.0"], 
                  check=True)
    print("   ✅ TensorFlow 2.14.0")
except subprocess.CalledProcessError as e:
    print(f"   ❌ TensorFlow: {e}")

# DIŞER GELİŞMİŞ PAKETLER
print("\n🚀 Gelişmiş paketler yükleniyor...")
advanced_packages = [
    'catboost>=1.2.0',
    'plotly>=5.17.0',
    'optuna>=3.0.0',
    'PyWavelets>=1.4.1',
    'nolds>=0.5.2'
]

for package in advanced_packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      check=True)
        print(f"   ✅ {package}")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ {package}: {e}")

# AUTOGLUON KURULUMU (En sona)
print("\n🤖 AutoGluon yükleniyor...")
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "autogluon>=0.8.0"], 
                  check=True)
    print("   ✅ AutoGluon")
except subprocess.CalledProcessError as e:
    print(f"   ❌ AutoGluon: {e}")

# Proje klonlama
print("\n📥 Proje klonlanıyor...")
if not os.path.exists('jetxpredictor'):
    !git clone https://github.com/onndd/jetxpredictor.git
    print("✅ Proje başarıyla klonlandı!")
else:
    print("✅ Proje zaten mevcut")

# Dizin kontrolü
print("\n📂 Dizin kontrolü yapılıyor...")
print(f"Mevcut dizin: {os.getcwd()}")

if os.path.exists('jetx_data.db'):
    print("✅ Proje kök dizinindeyiz!")
elif os.path.exists('../jetx_data.db'):
    print("📁 Bir üst dizine çıkılıyor...")
    os.chdir('..')
    print(f"✅ Proje kök dizinine geçildi: {os.getcwd()}")
elif os.path.exists('jetxpredictor/jetx_data.db'):
    print("📁 jetxpredictor klasörüne giriliyor...")
    os.chdir('jetxpredictor')
    print(f"✅ Proje kök dizinine geçildi: {os.getcwd()}")
else:
    print("⚠️ UYARI: jetx_data.db bulunamadı!")
    print(f"📂 Mevcut dizin içeriği: {os.listdir('.')}")

# Models klasörünü kontrol et ve oluştur
if not os.path.exists('models'):
    os.makedirs('models')
    print("✅ models/ klasörü oluşturuldu")
else:
    print("✅ models/ klasörü mevcut")

# Versiyon kontrolü
print("\n🔍 Versiyon kontrolü yapılıyor...")
try:
    import torch
    print(f"   PyTorch: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"   CUDA: {torch.version.cuda}")
        print(f"   GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("   CUDA: Mevcut değil")
except ImportError:
    print("   PyTorch: Yüklü değil")

try:
    import tensorflow as tf
    print(f"   TensorFlow: {tf.__version__}")
except ImportError:
    print("   TensorFlow: Yüklü değil")

try:
    import catboost
    print(f"   CatBoost: {catboost.__version__}")
except ImportError:
    print("   CatBoost: Yüklü değil")

try:
    import autogluon
    print(f"   AutoGluon: {autogluon.__version__}")
except ImportError:
    print("   AutoGluon: Yüklü değil")

prep_time = time.time() - start_time
print(f"\n✅ Hazırlık tamamlandı! ({prep_time/60:.1f} dakika)")
print(f"📂 Çalışma dizini: {os.getcwd()}")
print("="*80)

## 📊 Adım 2: Veri Yükleme ve Hazırlık

Veritabanından veri yüklenip train/val/test olarak bölünür.

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append(os.getcwd())

from category_definitions import CategoryDefinitions, FeatureEngineering
from utils.multi_scale_window import split_data_preserving_order

print("="*80)
print("📊 VERİ YÜKLEME")
print("="*80)

conn = sqlite3.connect('jetx_data.db')
data = pd.read_sql_query("SELECT value FROM jetx_results ORDER BY id", conn)
conn.close()

all_values = data['value'].values
print(f"✅ {len(all_values):,} veri yüklendi")

# Time-series split
train_data, val_data, test_data = split_data_preserving_order(
    all_values, train_ratio=0.70, val_ratio=0.15
)

print(f"✅ Train: {len(train_data):,}")
print(f"✅ Val:   {len(val_data):,}")
print(f"✅ Test:  {len(test_data):,}")
print("="*80)

## 🤖 Adım 3: Simple RL Agent Training (TensorFree)

TensorFlow/PyTorch sorunlarından kaçınmak için basit versiyon.

In [ ]:
print("="*80)
print("🤖 SIMPLE RL AGENT TRAINING (TensorFree)")
print("="*80)

!python notebooks/train_rl_agent_simple.py

# Sonuçları yükle
try:
    with open('models/simple_rl_agent_info.json', 'r') as f:
        rl_results = json.load(f)
    print("\n✅ Simple RL Agent Tamamlandı!")
    print(f"📊 Doğruluk: {rl_results.get('final_accuracy', 0):.2%}")
    print(f"📊 Action Distribution: {rl_results.get('action_distribution', [0, 0, 0, 0])}")
except Exception as e:
    print(f"\n⚠️ RL Agent info yüklenemedi: {e}")
    rl_results = None

## 🚀 Adım 4: CatBoost Training (PyTorch Free)

Gradient boosting uzmanı CatBoost modeli eğitimi.

In [ ]:
print("="*80)
print("🚀 CATBOOST TRAINING")
print("="*80)

!python notebooks/jetx_CATBOOST_TRAINING_MULTISCALE.py

with open('models/catboost_multiscale/model_info.json', 'r') as f:
    catboost_results = json.load(f)

print("\n✅ CatBoost Tamamlandı!")

## 🎯 Adım 5: Sonuçları Kaydetme ve İndirme

Tüm modeller ZIP'lenir ve indirilir. Google Drive'a yedekleme yapılır.

In [ ]:
import shutil

print("="*80)
print("💾 SONUÇLAR KAYDEDİLİYOR")
print("="*80)

# JSON sonuçları
final_results = {
    'metadata': {
        'version': '7.3',
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'fix': 'Dependency conflicts resolved'
    },
    'models': {
        'simple_rl_agent': rl_results if rl_results else None,
        'catboost': catboost_results if 'catboost_results' in locals() else None
    }
}

with open('models/fixed_models_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("✅ JSON sonuçları kaydedildi")

# ZIP oluştur
zip_filename = f'jetx_fixed_models_v7_{datetime.now().strftime("%Y%m%d_%H%M")}'
shutil.make_archive(zip_filename, 'zip', 'models')

print(f"\n✅ ZIP oluşturuldu: {zip_filename}.zip")

# Drive'a yedekle
if drive_path:
    try:
        drive_zip = f"{drive_path}/{zip_filename}.zip"
        shutil.copy(f"{zip_filename}.zip", drive_zip)
        print(f"✅ Drive'a yedeklendi: {drive_zip}")
    except Exception as e:
        print(f"⚠️ Drive yedekleme hatası: {e}")

# İndirme
try:
    from google.colab import files
    files.download(f'{zip_filename}.zip')
    print(f"\n✅ Dosya indiriliyor...")
    print("\n📌 İNDİRME TALİMATLARI:")
    print("="*80)
    print("1. ZIP dosyasını masaüstünde açın")
    print("2. İçindeki tüm dosyaları jetxpredictor/models/ klasörüne kopyalayın")
    print("3. Terminal'de: streamlit run app.py")
    print("4. '✅ Model yüklendi ve hazır!' mesajını görmelisiniz")
    print("="*80)
except:
    print(f"\n📁 Dosya konumu: {os.getcwd()}/{zip_filename}.zip")
    print("Manuel indirme: Dosyalar > Sağ tık > İndir")

print("\n" + "="*80)
print("✅ TÜM İŞLEMLER TAMAMLANDI!")
print("🔧 Dependency Conflict Fixed!")
print("="*80)

## 🎉 Özet

Bu notebook ile:

- ✅ **PyTorch/TorchAudio conflict çözüldü**
- ✅ **Versiyon uyumluluğu sağlandı** 
- ✅ **TensorFree RL Agent** ile TensorFlow sorunları önlendi
- ✅ **CatBoost modeli** başarıyla eğitildi
- ✅ **Google Drive yedekleme**
- ✅ **Kolay indirme sistemi**

Artık dependency çakışması olmadan modellerinizi eğitebilirsiniz! 🚀